## KaraFan

The BEST music separation model with help of **A.I** ... to my ears ! 👂👂

#### The Story of this project, Explanations, and Thanks are on my ... **[![GitHub](https://img.shields.io/badge/GitHub-100000?style=for-the-badge&logo=github&&labelColor=333&logoColor=white&color=ff0080)](https://github.com/Captain-FLAM/KaraFan)**

Take your time to read it, it's worth it ! 😁

If you want to encourage me to give more time to improve this project,  
you can make a donation or become one of my patrons :



In [ ]:
import os, sys

#****************************************************************
# If you still have GPU memory errors, you can try to
# uncomment the following line, but I'm not sure it will work...
# Anyway, following the Nvidia documentation,
# they recommend to set this environment variable to 512 MB :

# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = "max_split_size_mb:512"
#****************************************************************

Gdrive = ""; isColab = False
# Are we on Google Colab ?
try:
	from google.colab import drive
	isColab = True
	Gdrive  = "/content/Gdrive/MyDrive"
	drive.mount("/content/Gdrive", timeout_ms=10000)  # 10 seconds
except:
	# We are on a PC : Get the current path and remove last part (KaraFan)
	parts  = os.getcwd().split(os.path.sep)[:-1]
	Gdrive = os.path.sep.join(parts)

if isColab and not os.path.exists(Gdrive):
	print("****  You have denied access to your Google Drive !  ****")
	exit(0)

# Mandatory to import App modules
sys.path.insert(0, os.path.join(Gdrive, "KaraFan"))

import App.gui

## 😎 This is it !

In [ ]:
App.gui.Run(Gdrive, isColab)